In [1]:
import re
import sys
from uuid import uuid1
from time import sleep
from pprint import pprint
from pathlib import Path
from tempfile import gettempdir
from threading import Thread
from functools import partial
from subprocess import Popen, PIPE

from tqdm.notebook import tqdm
import numpy as np
import optuna
import pandas as pd
import matplotlib.pyplot as plt

FILENAME = "main.cpp"

In [2]:
# ここでエラー出力は何かおかしいかもしれない


# パラメータ抽出
with open(FILENAME) as f:
    answer = f.read()


variables_optimize = []

for left, value, right, options in re.findall(r"^([^/\n]*=\s*)(.+?)(\s*;\s*//(?:.*\W)?OPTIMIZE(\W.*))$", answer, re.MULTILINE):
    name = left.replace("=", "").strip().split()[-1]
    
    searched = re.search(r".*\[(?P<min>.*),(?P<max>.*)\].*", options)
    if searched:
        min_value = max_value = None
        try:
            min_value = eval(searched.group("min"))
            max_value = eval(searched.group("max"))
            assert min_value <= max_value
        except Exception as e:
            print(f"searched={searched}", file=sys.stderr)
            print(e, file=sys.stderr)
            continue
        log = "LOG" in options  # 雑、直したほうが良い
        if type(min_value) != type(max_value):
            print(f"searched={searched}", file=sys.stderr)
            print("types not matched", file=sys.stderr)
            continue
        if isinstance(min_value, int):
            method = "suggest_int"
        elif isinstance(min_value, float):
            method = "suggest_float"
        else:
            print(f"searched={searched}", file=sys.stderr)
            print(f"unknown type ({type(min_value)})", file=sys.stderr)
            continue
        variables_optimize.append({
            "name": name,
            "method": method,
            "min": min_value,
            "max": max_value,
            "log": log,
            "left": left,
            "right": right,
        })
    elif searched := re.search(r".*\{(?P<choices>.*?)\}.*", options):
        choices = list(map(lambda x: x.strip(), searched.group("choices").split(",")))
        variables_optimize.append({
            "name": name,
            "method": "suggest_categorical",
            "choices": choices,
            "left": left,
            "right": right,
        })
    else:
        print(f"searched={searched}", file=sys.stderr)
        print(f"pattern was matched but options are incorrect.", file=sys.stderr)

print(len(variables_optimize), "variables were found.")
if globals().get("pd"):
    display(pd.DataFrame(variables_optimize))
else:
    pprint(variables_optimize)

10 variables were found.


,name,method,min,max,log,left,right
0,kErase,suggest_int,1.000,5.0,False,static constexpr auto kErase =,"; // OPTIMIZE [1, 5]"
1,kRadius,suggest_int,2.000,6.0,False,static constexpr auto kRadius =,"; // OPTIMIZE [2, 6]"
2,kAnnealingA,suggest_float,-15.000,15.0,False,static constexpr auto kAnnealingA =,"; // OPTIMIZE [-15.0, 15.0]"
3,kAnnealingB,suggest_float,0.000,3.0,False,static constexpr auto kAnnealingB =,"; // OPTIMIZE [0.0, 3.0]"
4,kAnnealingStart,suggest_float,1.000,100.0,True,static constexpr auto kAnnealingStart =,"; // OPTIMIZE LOG [1.0, 100.0]"
5,kSkipRatio,suggest_float,0.200,0.8,False,static constexpr auto kSkipRatio =,"; // OPTIMIZE [0.2, 0.8]"
6,kTargetDeterminationTrials,suggest_int,1.000,20.0,True,static constexpr auto kTargetDeterminationTria...,"; // OPTIMIZE LOG [1, 20]"
7,kAttractionRatio,suggest_float,0.010,0.9,True,static constexpr auto kAttractionRatio =,"; // OPTIMIZE LOG [0.01, 0.9]"
8,kMaxAttractionDistance,suggest_int,4.000,99.0,True,static constexpr auto kMaxAttractionDistance =,"; // OPTIMIZE LOG [4, 99]"
9,kStartAttraction,suggest_float,0.001,0.9,True,static constexpr auto kStartAttraction =,"; // OPTIMIZE LOG [0.001, 0.9]"


In [3]:
def escape(string):  # 正規表現の中でそのまま使いたい文字列をエスケープ
    res = !echo '{string}' | sed -e 's/[]\/$*.^[]/\\&/g'
    return res[0]

def escape_sed(string):  # sed の置換後の文字列用のエスケープ
    res = !echo '{string}' | sed -e 's/[\/&]/\\&/g'
    return res[0]

def read_stream(name, in_file, out_file):
    for line in in_file:
        print(f"[{name}] {line.strip()}", file=out_file)

def run(cmd, name):
    proc = Popen(cmd, stdout=PIPE, stderr=PIPE, universal_newlines=True, shell=isinstance(cmd, str))
    stdout_thread = Thread(target=read_stream, args=(name, proc.stdout, sys.stdout))
    stderr_thread = Thread(target=read_stream, args=(name, proc.stderr, sys.stderr))
    stdout_thread.start()
    stderr_thread.start()
    proc.wait()
    return proc

def objective(trial, in_dir, work_dir):
    n_internal_parallel = 3
    
    index_parallel = f"{trial.number:04d}"
    print(f"{index_parallel=}")
    
    work_dir = Path(work_dir)
    directory_input = Path(in_dir)  # 中のすべてのファイルに対して実行される
    #parameters_changed_filename = Path(gettempdir()) / str(uuid1())
    parameters_changed_filename = work_dir / f"{index_parallel}_{FILENAME}"
    
    run(["mkdir", f"{work_dir / index_parallel}_out"], "mkdir")
    run(["mkdir", f"{work_dir / index_parallel}_score"], "mkdir")
    
    # ファイル作成
    run(f"cp {FILENAME} {parameters_changed_filename}", "cp")
    sed_options = [f"-i {parameters_changed_filename}"]
    for variable in variables_optimize:
        if variable["method"] == "suggest_categorical":
            val =  trial.suggest_categorical(variable["name"], variable["choices"])
        else:
            val = getattr(trial, variable["method"])(variable["name"], variable["min"], variable["max"], log=variable["log"])
        left = variable["left"]
        right = variable["right"]
        sed_options.append(f"""-e 's/^{escape(left)}.*{escape(right)}$/{escape_sed(left)}{val}{escape_sed(right)}/'""")
    command_sed = f"sed {' '.join(sed_options)}"
    #print(command_sed)
    run(command_sed, "sed")
    
    # コンパイル
    command_compile = f"g++ {parameters_changed_filename} -std=gnu++17 -O2 -DONLINE_JUDGE -o {parameters_changed_filename}.out"
    #print(command_compile)
    run(command_compile, "compile")
    # 実行・採点コマンド (@ はファイル名)
    command_exec = (
        #f"./a.out < ./tools/in/{i:04d}.txt > {out_file} && ./tools/target/release/vis ./tools/in/{i:04d}.txt {out_file}"
        #f"../tools/target/release/tester $(pwd)/{parameters_changed_filename}.out < {directory_input}/@ 2>&1 | grep Score"
        #f"../tools/target/release/tester $(pwd)/{parameters_changed_filename}.out < {directory_input}/@ 2>&1 | grep Score | sed -E s/[^0-9]+// > ./{index_parallel}_score/@;"
        #f"cargo run --release --manifest-path ../tools/Cargo.toml --bin tester {directory_input}/@ $(pwd)/{parameters_changed_filename}.out 2>&1 | grep Score | sed -E s/[^0-9]+// > ./{index_parallel}_score/@;"
        f"./{parameters_changed_filename}.out < {directory_input}/@ > {work_dir / index_parallel}_out/@;"
        f"../tools/target/release/vis {directory_input}/@ {work_dir / index_parallel}_out/@ 2> /dev/null > {work_dir / index_parallel}_score/@;"
    )
    # 並列実行 (sed はパスのディレクトリ部分を消してファイル名にしてる)
    run(f"find {directory_input}/* | sed 's!^.*/!!' | xargs -I@ -P {n_internal_parallel} sh -c '{command_exec}'", "exec")
    
    # 集計
    scores = []
    for file_path in Path(f"{work_dir / index_parallel}_score/").iterdir():  
        with open(file_path) as f:
            scores.append(int(f.readline().strip().split()[-1]))
    mean_score = sum(scores) / len(scores)
    
    # 後始末
    run(f"rm -rf {work_dir / index_parallel}_out", "rm")
    run(f"rm -rf {work_dir / index_parallel}_score", "rm")
    #run(f"rm {parameters_changed_filename}", "rm")
    run(f"rm {parameters_changed_filename}.out", "rm")
    
    return mean_score

In [4]:
K = 5
N_CLASS = 3

in_dir = Path(f"in_{K}_{N_CLASS}")
work_dir = Path(f"work_{K}_{N_CLASS}")

study_name = f"study_{K}_{N_CLASS}"
storage_path = work_dir / "study.db"
storage = f"sqlite:///{storage_path}"
study = optuna.create_study(storage=storage, load_if_exists=True, study_name=study_name, direction="maximize")

def callback(study, trial):
    try:
        index_parallel = f"{trial.number:04d}"
        parameters_changed_filename = work_dir / f"{index_parallel}_{FILENAME}"
        if study.best_value == trial.value:
            print(f"Updated! {study.best_value}")
            !cp {parameters_changed_filename} {work_dir / "best_parameters.cpp"}
        !rm {parameters_changed_filename}
    except:
        print(":(")


[I 2022-08-15 21:31:56,811] A new study created in RDB with name: study_5_3


In [ ]:
study.optimize(
    partial(objective, in_dir=in_dir, work_dir=work_dir),
    n_trials=3000,
    timeout=86400,
    callbacks=[callback]
)

index_parallel='0000'


[I 2022-08-15 21:36:19,023] Trial 0 finished with value: 7579.094420600858 and parameters: {'kErase': 2, 'kRadius': 5, 'kAnnealingA': -9.536895883841439, 'kAnnealingB': 0.24245400197956568, 'kAnnealingStart': 1.1931052975488052, 'kSkipRatio': 0.49685231929885276, 'kTargetDeterminationTrials': 14, 'kAttractionRatio': 0.020361029525395192, 'kMaxAttractionDistance': 4, 'kStartAttraction': 0.004335993212833265}. Best is trial 0 with value: 7579.094420600858.


Updated! 7579.094420600858
index_parallel='0001'


[I 2022-08-15 21:40:41,619] Trial 1 finished with value: 5177.0085836909875 and parameters: {'kErase': 4, 'kRadius': 5, 'kAnnealingA': 4.60192309053523, 'kAnnealingB': 1.1223821145394721, 'kAnnealingStart': 64.2851996465741, 'kSkipRatio': 0.29332554899903046, 'kTargetDeterminationTrials': 18, 'kAttractionRatio': 0.04757664916189116, 'kMaxAttractionDistance': 47, 'kStartAttraction': 0.05559280237809573}. Best is trial 0 with value: 7579.094420600858.


index_parallel='0002'


...

[I 2022-08-16 02:24:53,567] Trial 66 finished with value: 8104.772532188841 and parameters: {'kErase': 1, 'kRadius': 3, 'kAnnealingA': 13.03942997249276, 'kAnnealingB': 2.8616028759168506, 'kAnnealingStart': 9.999470299419839, 'kSkipRatio': 0.618184566559247, 'kTargetDeterminationTrials': 6, 'kAttractionRatio': 0.5700773766255924, 'kMaxAttractionDistance': 83, 'kStartAttraction': 0.35379020939250894}. Best is trial 52 with value: 8510.45922746781.


index_parallel='0067'


[I 2022-08-16 02:29:15,787] Trial 67 finished with value: 7802.798283261803 and parameters: {'kErase': 2, 'kRadius': 4, 'kAnnealingA': -6.498327064445871, 'kAnnealingB': 0.13307613669533547, 'kAnnealingStart': 4.064590810769772, 'kSkipRatio': 0.5961401176304144, 'kTargetDeterminationTrials': 9, 'kAttractionRatio': 0.26107530856140054, 'kMaxAttractionDistance': 15, 'kStartAttraction': 0.002064166141078944}. Best is trial 52 with value: 8510.45922746781.


index_parallel='0068'


[I 2022-08-16 02:33:37,897] Trial 68 finished with value: 7905.412017167382 and parameters: {'kErase': 1, 'kRadius': 3, 'kAnnealingA': -4.668587719509699, 'kAnnealingB': 1.0092734112030235, 'kAnnealingStart': 19.503786485282827, 'kSkipRatio': 0.6735952406300904, 'kTargetDeterminationTrials': 5, 'kAttractionRatio': 0.18248238191871674, 'kMaxAttractionDistance': 19, 'kStartAttraction': 0.14468305677370707}. Best is trial 52 with value: 8510.45922746781.
